## Introduction

<p>One useful data point in detecting fraud is the account history of a customer.
For an account, we receive notification of purchases and sometimes reports of fraud. 
Typically, a prior report of fraud for an account would increase the perceied risk of fraud on future transactions.</p>

<p>Similarly, a history of non-fraudulent purchases for an account would decrease the risk of fraud.
A credit card holder has 90 days to report any fraudulent transactions with the card.
So, if an account has purchases over 90 days old and no reports of fraud, we assume that these older purchases were not fraudulent.</p>


## Describing the Problem

<p>The purpose of this programming problem is to determine the status of a customer account history at the time a new purchase is made.</p>

<p>The input is a sequence of customer account events, in chronological order. Each event has three fields, all of which are of string type</p> 

<code>&lt;DATE&gt;, &lt;CUSTOMER_ACCOUNT_ID&gt;, &lt;EVENT_TYPE&gt;</code>
    

<p>For example:</p>

<ul style="list-style-type:none;">
    <li><code>2015-01-01,joe@signifyd.com,PURCHASE</code></li>
    <li><code>2015-02-01,fraudster@fraud.com,FRAUD_REPORT</code></li>
</ul>

<div>
<p>There are two event types</p>
    <ol>
        <li><code>PURCHASE</code> - indicates a purchase by this customer account on the specified date.</li>
        <li><code>FRAUD_REPORT</code> - indicates we received a report of fraud associated with this customer account. The specified date is that date that we received the report, not the date the fraud was committed.</li>
    </ol>
</div>

<p>For each event <code>PURCHASE</code>, we are interested in a summary of the customer account history based on prior events. The summary consists of the date of the summary, the customer account ID, and a status.</p>
<br />
<div>
<p>There are four possible values for the status of the customer account history:</p>
    <ol>
        <li><code>NO_HISTORY</code> - there are no prior events for this customer account</li>
        <li><code>FRAUD_HISTORY</code> - we have at least one event <code>FRAUD_REPORT</code> for this customer account</li>
        <li><code>GOOD_HISTORY</code> - customer account has no <code>FRAUD_REPORT</code> and at least one prior that <code>PURCHASE</code> is more than 90 days old</li>
        <li><code>UNCONFIRMED_HISTORY</code> - customer account has no <code>FRAUD_REPORT</code> and at least one prior <code>PURCHASE</code> but no <code>PURCHASE</code> over 90 days old</li>
    </ol>
</div>

<p>For accounts with <code>FRAUD_HISTORY</code>, <code>GOOD_HISTORY</code>, and <code>UNCONFIRMED_HISTORY</code>, the status also contains a count of relevant events.</p>
<ul style="list-style-type:none;">
    <li><code>FRAUD_HISTORY</code> - count of <code>FRAUD_REPORT</code></li>
    <li><code>GOOD_HISTORY</code> - count of prior <code>PURCHASE</code> over 90 days old</li>
    <li><code>UNCONFIRMED_HISTORY</code> - count of prior <code>PURCHASE</code></li>
</ul>
<p>The output is expected to be in the same order as the input</p>

## Sample Input and Output

<p>For the following input:</p>

<ul style="list-style-type:none;">
    <li><code>2015-01-01,joe@signifyd.com,PURCHASE</code></li>
    <li><code>2015-02-01,fraudster@fraud.com,FRAUD_REPORT</code></li>
    <li><code>2015-02-03,fraudster@fraud.com,FRAUD_REPORT</code></li>
    <li><code>2015-02-10,joe@signifyd.com,PURCHASE</code></li>
    <li><code>2015-02-14,fraudster@fraud.com,PURCHASE</code></li>
    <li><code>2015-03-15,joe@signifyd.com,PURCHASE</code></li>
    <li><code>2015-05-01,joe@signifyd.com,PURCHASE</code></li>
    <li><code>2015-10-01,joe@signifyd.com,PURCHASE</code></li>
</ul>
<br />
<ul style="list-style-type:none;">
    <li><code>2015-01-01,joe@signifyd.com,NO_HISTORY</code></li>
    <li><code>2015-02-10,joe@signifyd.com,UNCONFIRMED_HISTORY:1</code></li>
    <li><code>2015-02-14,fraudster@fraud.com,FRAUD_HISTORY:2</code></li>
    <li><code>2015-03-15,joe@signifyd.com,UNCONFIRMED_HISTORY:2</code></li>
    <li><code>2015-05-01,joe@signifyd.com,GOOD_HISTORY:1</code></li>
    <li><code>2015-10-01,joe@signifyd.com,GOOD_HISTORY:4</code></li>
</ul>

In [1]:
from csv import reader

# open file in read mode

with open('input.csv', 'r') as read_obj:
    # pass file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    
    # iterate over each row in the csv using reader object
    for row in csv_reader:
        # row variable is a list that represents a row in csv
        print(row)

['2015-01-01', 'joe@signifyd.com', 'PURCHASE']
['2015-02-01', 'fraudster@fraud.com', 'FRAUD_REPORT']
['2015-02-03', 'fraudster@fraud.com', 'FRAUD_REPORT']
['2015-02-10', 'joe@signifyd.com', 'PURCHASE']
['2015-02-14', 'fraudster@fraud.com', 'PURCHASE']
['2015-03-15', 'joe@signifyd.com', 'PURCHASE']
['2015-05-01', 'joe@signifyd.com', 'PURCHASE']
['2015-10-01', 'joe@signifyd.com', 'PURCHASE']


<i>Use pandas to store this input into a dataframe</i>

In [2]:
import pandas as pd

df = pd.read_csv('input.csv', header=None)
df

,0,1,2
0,2015-01-01,joe@signifyd.com,PURCHASE
1,2015-02-01,fraudster@fraud.com,FRAUD_REPORT
2,2015-02-03,fraudster@fraud.com,FRAUD_REPORT
3,2015-02-10,joe@signifyd.com,PURCHASE
4,2015-02-14,fraudster@fraud.com,PURCHASE
5,2015-03-15,joe@signifyd.com,PURCHASE
6,2015-05-01,joe@signifyd.com,PURCHASE
7,2015-10-01,joe@signifyd.com,PURCHASE


<i>Let's add a header for clarity and view the dataframe head</i>

In [3]:
# considering the email as the customer account ID
df.set_axis(['date','id','event'],axis=1,inplace=True)
df['date'] = pd.to_datetime(df['date'])

df.head()

,date,id,event
0,2015-01-01,joe@signifyd.com,PURCHASE
1,2015-02-01,fraudster@fraud.com,FRAUD_REPORT
2,2015-02-03,fraudster@fraud.com,FRAUD_REPORT
3,2015-02-10,joe@signifyd.com,PURCHASE
4,2015-02-14,fraudster@fraud.com,PURCHASE


<i>Now let's view the dataframe as a whole</i>

In [4]:
df

,date,id,event
0,2015-01-01,joe@signifyd.com,PURCHASE
1,2015-02-01,fraudster@fraud.com,FRAUD_REPORT
2,2015-02-03,fraudster@fraud.com,FRAUD_REPORT
3,2015-02-10,joe@signifyd.com,PURCHASE
4,2015-02-14,fraudster@fraud.com,PURCHASE
5,2015-03-15,joe@signifyd.com,PURCHASE
6,2015-05-01,joe@signifyd.com,PURCHASE
7,2015-10-01,joe@signifyd.com,PURCHASE


<i>We can now access cell values by index and column</i>

In [5]:
a = df.shape
b = df.loc[2, 'id']
c = list(df['date'])
d = df.loc[5]
e = d.id
f = d.event

In [6]:
print('________________\n')

print('The dimensions of our dataframe matrix in rows and columns: \n')
print(a)

print('________________\n')

print('The email at index [2], or the third element in event history: \n')
print(b)

print('________________\n')

print('A list of all dates in event history: \n')
print(c)

print('________________\n')

print('The information (by column) of index [5], or the sixth element in event history: \n')
print(d)

print('________________\n')

print('The email of element at index [5]: \n')
print(e)

print('________________\n')

print('The event type of element at index [5]: \n')
print(f)

print('________________\n')

________________

The dimensions of our dataframe matrix in rows and columns: 

(8, 3)
________________

The email at index [2], or the third element in event history: 

fraudster@fraud.com
________________

A list of all dates in event history: 

[Timestamp('2015-01-01 00:00:00'), Timestamp('2015-02-01 00:00:00'), Timestamp('2015-02-03 00:00:00'), Timestamp('2015-02-10 00:00:00'), Timestamp('2015-02-14 00:00:00'), Timestamp('2015-03-15 00:00:00'), Timestamp('2015-05-01 00:00:00'), Timestamp('2015-10-01 00:00:00')]
________________

The information (by column) of index [5], or the sixth element in event history: 

date     2015-03-15 00:00:00
id          joe@signifyd.com
event               PURCHASE
Name: 5, dtype: object
________________

The email of element at index [5]: 

joe@signifyd.com
________________

The event type of element at index [5]: 

PURCHASE
________________



<i>Let's refer back to the problem description and define some constants, variables, and parameters–</i>

<blockquote>
    <p>There are two event types</p>
    <ol>
        <li><code>PURCHASE</code> - indicates a purchase by this customer account on the specified date.</li>
        <li><code>FRAUD_REPORT</code> - indicates we received a report of fraud associated with this customer account. The specified date is that date that we received the report, not the date the fraud was committed.</li>
    </ol>
</blockquote>

<i>As well as–</i>

<blockquote>
    <p>There are four possible values for the status of the customer account history:</p>
    <ol>
        <li><code>NO_HISTORY</code> - there are no prior events for this customer account</li>
        <li><code>FRAUD_HISTORY</code> - we have at least one event <code>FRAUD_REPORT</code> for this customer account</li>
        <li><code>GOOD_HISTORY</code> - customer account has no <code>FRAUD_REPORT</code> and at least one prior that <code>PURCHASE</code> is more than 90 days old</li>
        <li><code>UNCONFIRMED_HISTORY</code> - customer account has no <code>FRAUD_REPORT</code> and at least one prior <code>PURCHASE</code> but no <code>PURCHASE</code> over 90 days old</li>
    </ol>
</blockquote>

<i>We should also take into account the information</i>

<blockquote>
    <p>For each event <code>PURCHASE</code>, we are interested in a summary of the customer account history based on prior events. The summary consists of the date of the summary, the customer account ID, and a status.</p>
</blockquote>

<blockquote>
    <p>For accounts with <code>FRAUD_HISTORY</code>, <code>GOOD_HISTORY</code>, and <code>UNCONFIRMED_HISTORY</code>, the status also contains a count of relevant events.</p>
<ul style="list-style-type:none;">
    <li><code>FRAUD_HISTORY</code> - count of <code>FRAUD_REPORT</code></li>
    <li><code>GOOD_HISTORY</code> - count of prior <code>PURCHASE</code> over 90 days old</li>
    <li><code>UNCONFIRMED_HISTORY</code> - count of prior <code>PURCHASE</code></li>
</ul>
</blockquote>

In [7]:
# event types

PURCHASE = 'PURCHASE'
FRAUD_REPORT = 'FRAUD_REPORT'


# customer account history status dictionary

CAH = dict.fromkeys(['NO_HISTORY','FRAUD_HISTORY',
                           'GOOD_HISTORY', 'UNCONFIRMED_HISTORY'])

# CAH keys created with empty values, initialize as lists

for key in CAH:
    CAH[key] = []
    
print(CAH)

{'NO_HISTORY': [], 'FRAUD_HISTORY': [], 'GOOD_HISTORY': [], 'UNCONFIRMED_HISTORY': []}


In [8]:
# create a list from restricted set of all customer account IDs
# initialize them to NO_HISTORY

for ID in list(set(df['id'])):
    CAH['NO_HISTORY'].append(ID)
        
print(CAH)

{'NO_HISTORY': ['joe@signifyd.com', 'fraudster@fraud.com'], 'FRAUD_HISTORY': [], 'GOOD_HISTORY': [], 'UNCONFIRMED_HISTORY': []}


In [9]:
# create a dataframe to hold an empty matrix to track activity

import numpy as np

IDlist = list(set(df['id']))
df2 = pd.DataFrame(np.zeros(shape=(3, len(IDlist))), columns=IDlist,
                   index=['fraud_history', 'good_history', 'unconfirmed_history'])

df2

,joe@signifyd.com,fraudster@fraud.com
fraud_history,0.0,0.0
good_history,0.0,0.0
unconfirmed_history,0.0,0.0


<i>Let's create a date dictionary with customer account IDs as keys, and pair them with lists holding dates of purchase</i>

In [10]:
datedict = dict.fromkeys(list(df['id']))

for key in datedict:
    datedict[key] = []

print(datedict)

{'joe@signifyd.com': [], 'fraudster@fraud.com': []}


In [11]:
def classify(index):
    
    ID = df.iloc[index].id
    event = df.iloc[index].event
    date = df.iloc[index].date
    
    if event == PURCHASE:
        updateDate(index)
        
        if ID in CAH['NO_HISTORY']:
            CAH['NO_HISTORY'].remove(ID)
            CAH['UNCONFIRMED_HISTORY'].append(ID)
            
    elif event == FRAUD_REPORT:
        
        if ID in CAH['NO_HISTORY']:
            CAH['NO_HISTORY'].remove(ID)
            CAH['FRAUD_HISTORY'].append(ID)
            
            # add fraud_history instance
            df2.loc['fraud_history', ID] += 1
            
        elif ID in CAH['FRAUD_HISTORY']:
            # add fraud_history instance
            df2.loc['fraud_history', ID] += 1

        elif ID in CAH['GOOD_HISTORY']:
            CAH['GOOD_HISTORY'].remove(ID)
            if ID not in CAH['FRAUD_HISTORY']:        
                CAH['FRAUD_HISTORY'].append(ID)
            
            # add fraud_history instance
            df2.loc['fraud_history', ID] += 1
            
        elif ID in CAH['UNCONFIRMED_HISTORY']:
            CAH['UNCONFIRMED_HISTORY'].remove(ID)
            if ID not in CAH['FRAUD_HISTORY']:
                CAH['FRAUD_HISTORY'].append(ID)
            
            # add fraud_history instance
            df2.loc['fraud_history', ID] += 1

    else:
        CAH['NO_HISTORY'].append(ID)

In [12]:
def printStatus(index):
    ID = df.iloc[index].id
    date = df.iloc[index].date
    
    if ID in CAH['NO_HISTORY']:
        print(str(date.date()) + ',' + ID + ',' + 'NO_HISTORY')
            
    elif ID in CAH['FRAUD_HISTORY']:
        print(str(date.date()) + ',' + ID + ',' + 'FRAUD_HISTORY:' 
              + str(df2.loc['fraud_history', ID]))

    elif ID in CAH['GOOD_HISTORY']:
        print(str(date.date()) + ',' + ID + ',' + 'GOOD_HISTORY:' 
              + str(df2.loc['good_history', ID]))
            
    elif ID in CAH['UNCONFIRMED_HISTORY']:
        print(str(date.date()) + ',' + ID + ',' + 'UNCONFIRMED_HISTORY:' 
              + str(df2.loc['unconfirmed_history', ID]))
        

In [13]:
import datetime

def updateDate(index):
    ID = df.iloc[index].id
    currdate = df.iloc[index].date

    if ID not in CAH['FRAUD_HISTORY']:
        if(len(datedict[ID]) > 0):
            for date in datedict[ID]:
                if(currdate - date) > datetime.timedelta(days=90):
                    df2.loc['unconfirmed_history', ID] -= 1
                    df2.loc['good_history', ID] += 1
                    datedict[ID].remove(date)

                    if ID not in CAH['GOOD_HISTORY']:
                        CAH['GOOD_HISTORY'].append(ID)
                    if ID in CAH['UNCONFIRMED_HISTORY']:
                        CAH['UNCONFIRMED_HISTORY'].remove(ID)

    printStatus(index)
    
    datedict[ID].append(currdate)            
    df2.loc['unconfirmed_history', ID] += 1
    
  

In [14]:
for i in range(len(list(df['date']))):
    classify(i)

2015-01-01,joe@signifyd.com,NO_HISTORY
2015-02-10,joe@signifyd.com,UNCONFIRMED_HISTORY:1.0
2015-02-14,fraudster@fraud.com,FRAUD_HISTORY:2.0
2015-03-15,joe@signifyd.com,UNCONFIRMED_HISTORY:2.0
2015-05-01,joe@signifyd.com,GOOD_HISTORY:1.0
2015-10-01,joe@signifyd.com,GOOD_HISTORY:3.0


<p>We seem to have encountered anomaly. There must be a discrepancy between what the datetime library considers a difference of 90 days; we will test this by viewing <code>datedict</code></p>

<p>We have already removed history of all purchases considered 'good' so as not to further interfere with our event tally dataframe in the <code>updateDate</code> function, so we will also analyze <code>df2</code> to confirm which entry is troubling us.</p> 

In [15]:
datedict

{'joe@signifyd.com': [Timestamp('2015-03-15 00:00:00'),
  Timestamp('2015-10-01 00:00:00')],
 'fraudster@fraud.com': [Timestamp('2015-02-14 00:00:00')]}

In [16]:
df2

,joe@signifyd.com,fraudster@fraud.com
fraud_history,0.0,2.0
good_history,3.0,0.0
unconfirmed_history,2.0,1.0


<p>This confirms our suspcions– there is a indeed a discrepancy in the performance of the library. An error occurs in the fifth index of the eigth (final) iteration of this loop
    
    <code>
        
        for date in datedict[ID]:
                if(currdate - date) > datetime.timedelta(days=90):
                    df2.loc['unconfirmed_history', ID] -= 1
                    df2.loc['good_history', ID] += 1
                    datedict[ID].remove(date)

                    if ID not in CAH['GOOD_HISTORY']:
                        CAH['GOOD_HISTORY'].append(ID)
                    if ID in CAH['UNCONFIRMED_HISTORY']:
                        CAH['UNCONFIRMED_HISTORY'].remove(ID)
    </code></p>
    
<p>When date <code>2015-03-15</code> is being compared to <code>2015-10-01</code>, the boolean returned from</p>

<code>if(currdate - date) > datetime.timedelta(days=90)</code>
 
<p>is not evaluated as true, thus does not go on to perform an addition and subtraction of the <code>good_history</code> and <code>unconfirmed_history</code> respectively, owing to the discrepant values in the event tally dataframe</p>

<p>For the current scope of the project, we will overlook this bug, however a compacted and optimized version of this code along with an export to csv is provided in the <a href="fraud-tracer-compact.ipynb"><code>fraud-tracer-compact</code></a> file.</p>